In [1]:
import sys
import os
from tqdm import tqdm
import random
import numpy as np
import math
# Add the parent directory to the sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

sys.path.append(parent_dir)

#import simtrain
from simtrain.sim_models_new import User_simmulation_Model
from simtrain import SETTINGS_POLIMI as SETTINGS, process_dat, Dataset

import torch
import torch.nn as nn

import ast

import paths
from os.path import join
import pytorch_warmup as warmup


In [2]:
num_items = 7
num_items_per_recom = 2
num_interaction_types = 2
recom_dim = 1
num_users = 11
min_inter = 2
max_inter = 4
state_size = SETTINGS.STATE_SIZE

In [4]:
'''
# generate toy data
data = []
for user in range(num_users):
    num_interactions_now = random.randint(a=min_inter, b=max_inter)
    new = {
        'item_ids': torch.randint(low=1, high=num_items, size=(num_interactions_now, 
        num_items_per_recom, recom_dim)).to(torch.float32),
        'timestamps': torch.sort(torch.FloatTensor(num_interactions_now).uniform_(0, 1.))[0].to(torch.float32),
        'interaction_types': torch.randint(low=0, high=num_interaction_types-1, 
        size=(num_interactions_now, num_items_per_recom)),
        "user_means": torch.randn((state_size), requires_grad=True).to(torch.float32),
        "user_vars_log": torch.randn((state_size), requires_grad=True).to(torch.float32),
    }
    data.append(new)


# Create the dataset
dataset = CustomDataset(data)

# Example usage with DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)# can only do batchsize 1
'''


'\n# generate toy data\ndata = []\nfor user in range(num_users):\n    num_interactions_now = random.randint(a=min_inter, b=max_inter)\n    new = {\n        \'item_ids\': torch.randint(low=1, high=num_items, size=(num_interactions_now, \n        num_items_per_recom, recom_dim)).to(torch.float32),\n        \'timestamps\': torch.sort(torch.FloatTensor(num_interactions_now).uniform_(0, 1.))[0].to(torch.float32),\n        \'interaction_types\': torch.randint(low=0, high=num_interaction_types-1, \n        size=(num_interactions_now, num_items_per_recom)),\n        "user_means": torch.randn((state_size), requires_grad=True).to(torch.float32),\n        "user_vars_log": torch.randn((state_size), requires_grad=True).to(torch.float32),\n    }\n    data.append(new)\n\n\n# Create the dataset\ndataset = CustomDataset(data)\n\n# Example usage with DataLoader\ndataloader = DataLoader(dataset, batch_size=1, shuffle=True)# can only do batchsize 1\n'

In [ ]:
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, data):
        """
        Args:
            data (list of dicts): Each dict contains 'timestamps', 'items', and 'labels' for a user.
        """
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        user_data = self.data[idx]
        timestamps = user_data['timestamps']
        items =  user_data['item_ids']
        labels = user_data['interaction_types']
        return timestamps, items, labels, user_data["user_means"], user_data["user_vars_log"], idx
    
    def Update_user_params(self, means_list, logvar_list, idx_list):
        #means_list.requires_grad = True
        #logvar_list.requires_grad = True
        
        #print(means_list)
        
        self.data[idx_list[0]]["user_means"] = means_list.tolist()
        self.data[idx_list[0]]["user_vars_log"] = logvar_list.tolist()
        #for means, logvar, idx in zip(means_list, logvar_list, idx_list):
        #    self.data[idx]["user_means"] = means
        #    self.data[idx]["user_vars_log"] = logvar


In [5]:
#create dataset from processed data
'''
train_dat, stg = process_dat.load_dat(paths.cw_stages['output_new']['train'], new_data=True)

print(stg)

def convert_string_to_double_list(s):
    return ast.literal_eval(s)

# Apply the custom function
train_dat['item_ids'] = train_dat['item_ids'].apply(convert_string_to_double_list)
train_dat['user_means'] = train_dat['user_means'].apply(convert_string_to_double_list)
train_dat['user_vars_log'] = train_dat['user_vars_log'].apply(convert_string_to_double_list)
train_dat['timestamps'] = train_dat['timestamps'].apply(convert_string_to_double_list)
train_dat['interaction_types'] = train_dat['interaction_types'].apply(convert_string_to_double_list)

print("len: ", len(train_dat))
list_of_dicts = train_dat.to_dict(orient='records')

train_dat.head()

'''

'\ntrain_dat, stg = process_dat.load_dat(paths.cw_stages[\'output_new\'][\'train\'], new_data=True)\n\nprint(stg)\n\ndef convert_string_to_double_list(s):\n    return ast.literal_eval(s)\n\n# Apply the custom function\ntrain_dat[\'item_ids\'] = train_dat[\'item_ids\'].apply(convert_string_to_double_list)\ntrain_dat[\'user_means\'] = train_dat[\'user_means\'].apply(convert_string_to_double_list)\ntrain_dat[\'user_vars_log\'] = train_dat[\'user_vars_log\'].apply(convert_string_to_double_list)\ntrain_dat[\'timestamps\'] = train_dat[\'timestamps\'].apply(convert_string_to_double_list)\ntrain_dat[\'interaction_types\'] = train_dat[\'interaction_types\'].apply(convert_string_to_double_list)\n\nprint("len: ", len(train_dat))\nlist_of_dicts = train_dat.to_dict(orient=\'records\')\n\ntrain_dat.head()\n\n'

In [6]:

checkpoint = torch.load(join(paths.dat, SETTINGS.filepaths_new['optimized_data']))
list_of_dicts = checkpoint['data']

In [8]:

dataset = CustomDataset(list_of_dicts[:50])
# Example usage with DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)# can only do batchsize 1

In [9]:
def test_timestamps(dataloader):
    smallest = float("inf")
    biggest = -1
    for batch in dataloader:
        timestamps, items, labels, means, var, idx = batch
        last = timestamps[0]
        smallest = min(smallest, last)
        biggest = max(biggest, timestamps[-1])
        for i in range(1,len(timestamps)):
            if timestamps[i] <= last:
                print("error, current: ", timestamps[i], "\tlast", last)
    print(smallest), print(biggest)
    return biggest

max_time = test_timestamps(dataloader)

tensor([0.0417], dtype=torch.float64)
tensor([69.3333], dtype=torch.float64)


In [10]:
for batch in dataloader:
    timestamps, items, labels, means, var, idx = batch
    print('Timestamps:', timestamps#, "\n dtype: ", timestamps.dtype
          )
    print('item_recom:', items#, "\n dtype: ", items.dtype
          )
    print('Labels:', labels#, "\n dtype: ", labels.dtype
          )
    print('means:', means#, "\n dtype: ", means.dtype
          )
    print('log_var:', var#, "\n dtype: ", var.dtype
          )
    break
    

Timestamps: [tensor([5.0833], dtype=torch.float64), tensor([6.3958], dtype=torch.float64), tensor([11.8125], dtype=torch.float64), tensor([28.0417], dtype=torch.float64), tensor([28.2708], dtype=torch.float64), tensor([40.1042], dtype=torch.float64), tensor([42.8542], dtype=torch.float64)]
item_recom: [[tensor([161]), tensor([199]), tensor([279]), tensor([12]), tensor([37]), tensor([84]), tensor([74]), tensor([132]), tensor([161]), tensor([284]), tensor([250]), tensor([161]), tensor([144]), tensor([165]), tensor([161]), tensor([249]), tensor([9]), tensor([144]), tensor([35]), tensor([256]), tensor([205]), tensor([66]), tensor([297]), tensor([263]), tensor([57]), tensor([217]), tensor([205]), tensor([89]), tensor([290]), tensor([199]), tensor([62]), tensor([79]), tensor([84]), tensor([74]), tensor([132]), tensor([162]), tensor([165]), tensor([161]), tensor([249]), tensor([19]), tensor([144]), tensor([192]), tensor([250]), tensor([42]), tensor([165]), tensor([195]), tensor([191]), tensor

In [11]:
def train_1_path(model, user_state, timestamps, items, labels, loss_func, num_classes, max_time, device, teacher_forcing=True):
    ''' expects batchsize of 1
    '''
    model.init_state(user_state)
    loss_base = 0.
    loss_intensity = 0.
    curr_time = 1e-10# because time 0 is used sadly
    N = len(timestamps)
    max_div_by_N = max_time/N
    for interaction_id in range(N):#[0]
        h = (timestamps[interaction_id] - curr_time).float()
        
        intensity = model.eval_intensity(h)#loss on that
        
        #try:
        model.evolve_state(h)
        #except Exception as e:
        #    print("delta: ",h , "\tnew: ", timestamps[interaction_id], "\told: ", curr_time)
        #    print(e)

        curr_time = h
        # no intensity for now
        y_pred = model.view_recommendations(items[interaction_id])# :,
        y_true = torch.as_tensor(labels[interaction_id])# :,
        y_true_onehot = nn.functional.one_hot(y_true, num_classes=num_classes).float()
        
        if teacher_forcing:
            model.jump(y_true_onehot)
        else:
            model.jump(y_pred)
        #loss += loss_func(y_true, y_pred)# for mse
        y_true = y_true.squeeze(0)
        y_pred = y_pred.squeeze(0)
        y_pred = nn.functional.log_softmax(y_pred)
        #print("true: ",y_true.shape, "\t predicted: ",y_pred.shape)
        #print(torch.unique(y_true))
        loss_base += loss_func(y_pred, y_true) # NLLL
        loss_intensity += -torch.log(intensity) + max_div_by_N*intensity
    
    return loss_base, loss_intensity


In [12]:
# parameter dicts
width= 10
user_state_dict = {"model_hyp": {"layer_width": [width, width, width]}}
intensity_state_dict = {"model_hyp": {"user_model_hyp": {"layer_width": [width, width, 3],
                                                         "noise": 0},
                                          "global_model_hyp": {"layer_width": [width, 3]}}
                            }
interaction_state_dict = {"model_hyp": {"layer_width": [width, width ,width]}
                            }
jump_state_dict = {"model_hyp": {"layer_width": [width, width]}
                        }

In [13]:
import torch.optim as optim

hyperparameter_dict = {"state_size": state_size, "state_model": user_state_dict, "num_interaction_outcomes": num_interaction_types,
                           "intensity_model": intensity_state_dict, "num_recom" : num_items_per_recom,
                            "recom_dim":recom_dim, "interaction_model": interaction_state_dict,
                            "jump_model": jump_state_dict}
model = User_simmulation_Model(hyperparameter_dict)

In [14]:
# load model
model.load_state_dict(torch.load(join(paths.dat, SETTINGS.filepaths_new['user_model'])))

<All keys matched successfully>

In [15]:
#loss_func = nn.functional.mse_loss
loss_func = nn.NLLLoss()

def kl_divergence(mu1, sigma1, mu2, sigma2):
    """
    Compute the KL divergence between two normal distributions N(mu1, sigma1^2) and N(mu2, sigma2^2).

    Args:
        mu1 (Tensor): Mean of the first distribution.
        sigma1 (Tensor): Standard deviation of the first distribution.
        mu2 (Tensor): Mean of the second distribution.
        sigma2 (Tensor): Standard deviation of the second distribution.

    Returns:
        Tensor: KL divergence.
    """
    kl_div = torch.log(sigma2 / sigma1) + ((sigma1 ** 2 + (mu1 - mu2) ** 2) / (2 * sigma2 ** 2)) - 0.5
    return kl_div

def kl_divergence_to_standard_normal(mu, sigma):
    """
    Compute the KL divergence from a normal distribution N(mu, sigma^2) to the standard normal distribution N(0, 1).

    Args:
        mu (Tensor): Mean of the normal distribution.
        sigma (Tensor): Standard deviation of the normal distribution.

    Returns:
        Tensor: KL divergence.
    """
    sigma2 = sigma ** 2
    kl_div = 0.5 * (sigma2 + mu ** 2 - torch.log(sigma2) - 1)
    return kl_div

def kl_loss(mu, sigma):
    return kl_divergence(mu, sigma, 0, 0.1)

In [30]:
def print_user_params(datadataloader, print_var = False, num_examples=5):
    i = 0
    for batch in dataloader:
        timestamps, item_recom, labels, means, logvar, idx = batch
        print("means: ", means)
        if (print_var):
            print("logvar: ", logvar)
        i+=1
        if i >= num_examples:
            return

In [41]:
def train(model, device, dataloader,num_epochs, state_size, loss_func, loss_func_kl, optimizer, num_classes, 
            logger, max_time,lr_scheduler, warmup_scheduler,
            kl_weight = 1, user_lr = None, log_step_size = 1, warmup_period=100):
    model.to(device)
    for epoch in tqdm(range(num_epochs)):  # Example: Number of epochs
        loss_all, loss_base, loss_kl, loss_intensity = 0, 0, 0, 0
        #print_user_params(dataloader)# see if values change
        for batch in dataloader:
            # Zero the gradients
            
            timestamps, item_recom, labels, means, logvar, idx = batch
            timestamps, means, logvar = torch.as_tensor(timestamps).to(device).float(), \
                 torch.as_tensor(means).to(device).float(), torch.as_tensor(logvar).to(device).float()  #  item_recom, labels, = item_recom.to(device), labels.to(device),
            
            #timestamps, item_recom, labels, means, logvar = torch.as_tensor(timestamps).to(device), \
            #    torch.as_tensor(item_recom).to(device), torch.as_tensor(labels).to(device), \
            #    torch.as_tensor(means).to(device), torch.as_tensor(logvar).to(device)
            means.requires_grad = True
            logvar.requires_grad = True
            #means.retain_grad()
            #logvar.retain_grad()

            variances = torch.exp(logvar)
            user_state = means + variances*torch.randn((1, state_size))
            #delta_from_previous = torch.cat([torch.zeros((timestamps.size(0),1)), timestamps[:,1:] - timestamps[:,:-1]], dim=1)
            
            curr_loss_base, curr_loss_intensity = train_1_path(model=model, user_state=user_state, timestamps=timestamps, items=item_recom, labels=labels,  
                         loss_func=loss_func, max_time=max_time, num_classes=num_classes, device=device)
            curr_loss_kl = kl_weight * torch.sum(loss_func_kl(means, variances))#.view(1,-1)
            
            curr_loss_all = curr_loss_kl + curr_loss_base+curr_loss_intensity
            curr_loss_all.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            #logging
            loss_all += curr_loss_all.item()
            loss_base += curr_loss_base.item()
            loss_kl += curr_loss_kl.item()
            loss_intensity += curr_loss_intensity.item()
            # maybe need to optim user_mean, user_var separate because of torch..
            if user_lr:
                torch.nn.utils.clip_grad_norm_(means, max_norm=1.0)
                torch.nn.utils.clip_grad_norm_(logvar, max_norm=1.0)
                with torch.no_grad():
                    means -= user_lr * means.grad
                    logvar -= user_lr * logvar.grad  
                #print(means)
                means.zero_grad()
                logvar.zero_grad()
                dataloader.dataset.Update_user_params(means.detach(), logvar.detach(), idx)
            
            #print("lr: ",optimizer.param_groups[0]['lr'])
            with warmup_scheduler.dampening():
                if warmup_scheduler.last_step + 1 >= warmup_period:
                    lr_scheduler.step()
            
            optimizer.zero_grad()
                
        if epoch % log_step_size == 0:
            logger(loss_all, loss_base, loss_kl, loss_intensity)
    logger(loss_all, loss_base, loss_kl, loss_intensity)# log at the end


In [42]:
def logging_func(loss_all, loss_base, loss_kl, loss_intensity):
    print("loss_all: ", loss_all, "\tloss_base: ",loss_base, "\tloss_kl: ",loss_kl, "\tloss_intensity: ",loss_intensity, "\tlog of the loss: ", math.log(loss_all))

In [43]:
num_epochs = 5
warmup_period = len(dataset)
num_steps = num_epochs*len(dataset) -warmup_period
num_iter_til_first_restart = num_steps//2


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

optimizer = optim.AdamW(model.parameters(), lr=0.01,
                        weight_decay=1e-8)
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=num_iter_til_first_restart, T_mult=1, eta_min=1e-4)

warmup_scheduler = warmup.LinearWarmup(optimizer, warmup_period)


Using device: cpu


In [44]:
print_user_params(dataloader, print_var = True)

means:  [tensor([0.1684], dtype=torch.float64), tensor([-0.2828], dtype=torch.float64), tensor([-0.0024], dtype=torch.float64), tensor([0.3760], dtype=torch.float64), tensor([0.0110], dtype=torch.float64), tensor([0.0295], dtype=torch.float64), tensor([-0.1031], dtype=torch.float64), tensor([-0.2597], dtype=torch.float64)]
logvar:  [tensor([-0.2200], dtype=torch.float64), tensor([-0.0910], dtype=torch.float64), tensor([0.0476], dtype=torch.float64), tensor([-0.0762], dtype=torch.float64), tensor([-0.0448], dtype=torch.float64), tensor([0.0836], dtype=torch.float64), tensor([0.0206], dtype=torch.float64), tensor([0.2200], dtype=torch.float64)]
means:  [tensor([0.1429], dtype=torch.float64), tensor([-0.2300], dtype=torch.float64), tensor([-0.0014], dtype=torch.float64), tensor([0.2801], dtype=torch.float64), tensor([0.1589], dtype=torch.float64), tensor([-0.0006], dtype=torch.float64), tensor([-0.0181], dtype=torch.float64), tensor([-0.1662], dtype=torch.float64)]
logvar:  [tensor([-0.21

In [45]:
train(model, dataloader=dataloader, num_epochs=num_epochs, device=device, loss_func=loss_func, 
                loss_func_kl=kl_divergence_to_standard_normal, kl_weight=1., user_lr=0.1,
                optimizer=optimizer, lr_scheduler=lr_scheduler, num_classes=num_interaction_types, 
                logger=logging_func,warmup_period=warmup_period,
                state_size=state_size,max_time=max_time, log_step_size=1, warmup_scheduler = warmup_scheduler,
                )

  0%|          | 0/5 [00:00<?, ?it/s]/home/thahit/anaconda3/envs/WW/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [28]:
print_user_params(dataloader, print_var = True)

means:  [tensor([0.0149], dtype=torch.float64), tensor([-0.0143], dtype=torch.float64), tensor([0.0468], dtype=torch.float64), tensor([0.0425], dtype=torch.float64), tensor([0.0993], dtype=torch.float64), tensor([0.0387], dtype=torch.float64), tensor([-0.0228], dtype=torch.float64), tensor([-0.0680], dtype=torch.float64)]
logvar:  [tensor([0.1817], dtype=torch.float64), tensor([-0.0555], dtype=torch.float64), tensor([-0.0278], dtype=torch.float64), tensor([-0.3957], dtype=torch.float64), tensor([0.0319], dtype=torch.float64), tensor([-0.1186], dtype=torch.float64), tensor([0.1501], dtype=torch.float64), tensor([0.3147], dtype=torch.float64)]
means:  [tensor([0.0865], dtype=torch.float64), tensor([-0.1777], dtype=torch.float64), tensor([0.0748], dtype=torch.float64), tensor([-0.1917], dtype=torch.float64), tensor([-0.0618], dtype=torch.float64), tensor([0.0318], dtype=torch.float64), tensor([0.0258], dtype=torch.float64), tensor([0.0558], dtype=torch.float64)]
logvar:  [tensor([0.0162],

In [ ]:
SETTINGS.filepaths['user_model']

'saved_models_polimi/accordion/user_model.h5'

In [ ]:
#save model
torch.save(model.state_dict(), join(paths.dat, SETTINGS.filepaths_new['user_model']))

In [ ]:
# save data(changes during training)
torch.save({
    'data': dataloader.dataset.data,
}, join(paths.dat, SETTINGS.filepaths_new['optimized_data'])# 'saved_models_polimi/data.h5'
)
